In [3]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_iris
iris = load_iris()

In [4]:
dir(iris) 

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [6]:
from sklearn.model_selection import train_test_split as tts

In [17]:
X_train, X_test, y_train, y_test = tts(iris.data, iris.target, test_size = 0.2)

In [11]:
model = SVC(C=50, gamma = 'auto', kernel='rbf')

In [18]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9666666666666667

In [20]:
from sklearn.model_selection import cross_val_score as cvs

In [28]:
cvs(SVC(C=50, gamma = 'auto', kernel='rbf'), iris.data, iris.target, cv = 5)

array([1.        , 0.96666667, 0.9       , 0.93333333, 1.        ])

In [34]:
cvs(SVC(C=10), iris.data, iris.target, cv = 5) # writting these again and again with diff SVC params is inconvinient so we make a for loop for this in below

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [35]:
kernels = ['linear', 'rbf']
C = [1, 10, 20]
average_scores = {}

for kval in kernels:
    for cval in C:
        c_scores = cvs(SVC(C=cval, gamma = 'auto', kernel= kval), iris.data, iris.target, cv = 5)
        average_scores[kval + '_' + str(cval)] = np.average(c_scores)
average_scores        

{'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668}

In [41]:
# we can also replace the above for loops with a single line 
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma = 'auto'),
                   {'C' :[1, 10, 20],
                     'kernel' :['linear', 'rbf']},
                   cv = 5, return_train_score = False)
clf.fit(iris.data, iris.target)
result =pd.DataFrame(clf.cv_results_) 

In [42]:
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000854,0.000257,0.000000,0.000000,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000599,0.000489,0.000422,0.000518,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000464,0.000290,0.000000,0.000000,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
3,0.000891,0.001328,0.000000,0.000000,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.000000,0.000000,0.001186,0.001891,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6
5,0.000200,0.000399,0.000600,0.000490,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5


In [45]:
result[['param_C', 'params', 'mean_test_score' ]]

,param_C,params,mean_test_score
0,1,"{'C': 1, 'kernel': 'linear'}",0.980000
1,1,"{'C': 1, 'kernel': 'rbf'}",0.980000
2,10,"{'C': 10, 'kernel': 'linear'}",0.973333
3,10,"{'C': 10, 'kernel': 'rbf'}",0.980000
4,20,"{'C': 20, 'kernel': 'linear'}",0.966667
5,20,"{'C': 20, 'kernel': 'rbf'}",0.966667


In [49]:
# if we dont have much computational power than we can use this below by giving it how many combinations we wanna try(n_iter),
# than it will radomly try 2 combinations and return us the values back
from sklearn.model_selection import RandomizedSearchCV
clf = RandomizedSearchCV(SVC(gamma = 'auto'),
                   {'C' :[1, 10, 20],
                     'kernel' :['linear', 'rbf']},
                   cv = 5, return_train_score = False,
                        n_iter = 2)
clf.fit(iris.data, iris.target)
result =pd.DataFrame(clf.cv_results_) 
result[['params','mean_test_score']]

,params,mean_test_score
0,"{'kernel': 'rbf', 'C': 1}",0.980000
1,"{'kernel': 'linear', 'C': 10}",0.973333


In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [54]:
# if we wanna try that for diff models then we can apply a for-loop here
scores = []
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv = 5, return_train_score = False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
    })
scores    

[{'model': 'svm',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'random_forest',
  'best_score': 0.9533333333333334,
  'best_params': {'n_estimators': 5}},
 {'model': 'logistic_regression',
  'best_score': 0.9666666666666668,
  'best_params': {'C': 5}}]